# Exam -  Tue 14, Jan 2021

**Scientific Programming - Data Science Master @ University of Trento**


## Download [exercises and solutions](_static/generated/sciprog-ds-2021-01-14-exam.zip)


## Part A - Witchcraft


<div class="alert alert-warning">
    
**NOTICE: this part of the exam was ported to** [softpython](https://en.softpython.org/projects/witchcraft/witchcraft-sol.html) **website**
    
There you can find a more curated version (notice it may be longer than here)
</div>    

The early sixteenth century saw a dramatic rise in awareness and terror of witchcraft in the troubled lands of [early modern Scotland](https://www.youtube.com/watch?v=4s9Hd8onAKQ): thousands of people were executed, imprisoned, tortured, banished, and had lands and possessions confiscated. Persecution took place in courts of law: you shall analyze the evidence gathered during those dark days.

Data source: Julian Goodare, Lauren Martin, Joyce Miller and Louise Yeoman, 'The Survey of Scottish Witchcraft'  http://www.shca.ed.ac.uk/witches/  (archived January 2003, accessed ’11/1/2016’).

In particular, we took [WDB_Case.csv](WDB_Case.csv) as published on [data.world](https://data.world/history/scottish-witchcraft), which contains cases brought against suspected witches, along with annotations by researchers, mostly as boolean fields.

The dataset has lots of columns, we show here only the relevant ones `Case_date`, `CaseCommonName`, `Suspects_text` and an excerpt of the many boolean columns:

In [1]:
import pandas as pd
case_df = pd.read_csv('WDB_Case.csv', encoding='UTF-8')
sel_case_df = case_df[['Case_date', 'CaseCommonName', 'Suspects_text', 'Demonic_p', 'Demonic_s', 'Maleficium_p','Maleficium_s', 'WitchesMeeting']]
sel_case_df[1986:1994]

Case_date      CaseCommonName  Suspects_text  Demonic_p  Demonic_s  \
1986   29/6/1649   3 unnamed witches            3.0          0          0   
1987   19/8/1590      Leslie,William            NaN          0          0   
1988        1679  McGuffock,Margaret            NaN          0          0   
1989        1679        Rae,Grissell            NaN          0          0   
1990        1679         Howat,Jonet            NaN          0          0   
1991  15/10/1673       McNicol,Janet            NaN          1          1   
1992    4/6/1674      Clerk,Margaret            NaN          0          0   
1993   29/7/1675       Hendrie,Agnes            NaN          0          1   

      Maleficium_p  Maleficium_s  WitchesMeeting  
1986             0             0               0  
1987             0             1               0  
1988             0             1               0  
1989             0             0               0  
1990             0             0               0  
1991             0             1               1  
1992             0             0               0  
1993             0             0               1

## A1 parse_bool_cols

Open Jupyter and start editing this notebook `exam-2021-01-14.ipynb`

Since boolean columns are so many, as a first step you will build a recognizer for them. 

* Consider a column as boolean if **ALL** of its values are either `0` or `1`
* Parse with [CSV DictReader](https://en.softpython.org/formats/formats-sol.html#Reading-as-dictionaries)

<div class="alert alert-warning">

**WARNING: Have you carefully read the text above?**
    
Most students during the exam didn't, and wrote algorithms which declare a column as boolean as soon as a `0` or a `1` are found and manually discarded columns which didn't fit their flawed logic (like `NamedIndividual`).
</div>    

To avoid getting bad grades for simple exercises, always ask yourself:
    
1. Am I sure about the results of my algorithm _without_ looking at the expected solution ?  In this case, it should be obvious that if you don't scan **all** cells in a column and you still declare it's boolean you are basically resorting to being lucky.
2. Am I putting _constants_ in the code (like `'NamedIndividual'`)? Whenever you have such urge, please ask first for permission
3. Is the exercise open to interpretation, maybe because it has so many possible weird cases and relative assertions, or is the text pretty clear? In this case the scope was quite definite, so you are expected to find a generic solution which could work with _any_ dataset.

**Example** (for full output see [expected_bool_cols.py](expected_bool_cols.py)):

```python
>>> bool_cols = get_bool_cols('WDB_Case.csv')
>>> print('Found', len(bool_cols), 'cols. EXCERPT:', ' '.join(bool_cols[:17]), '...')
Found 77 cols. EXCERPT: 
AdmitLesserCharge AggravatingDisease AnimalDeath AnimalIllness ClaimedBewitched ClaimedNaturalCauses ClaimedPossessed CommunalSex Consulting_p Consulting_s Cursing Dancing DemonicPact Demonic_p Demonic_possess_p Demonic_possess_s Demonic_s ...
```

In [2]:


def get_bool_cols(filename):
    """RETURN a sorted list of all the names of boolean columns"""
    

    import csv
    with open(filename, encoding='utf-8', newline='') as f:

        cols = set(next(csv.DictReader(f, delimiter=',')).keys())

    with open(filename, encoding='utf-8', newline='') as f:
        my_reader = csv.DictReader(f, delimiter=',')   # Notice we now used DictReader
        for diz in my_reader:
            for k in diz:

                if not (diz[k] == '0' or diz[k] == '1'):
                    if k in cols:
                        # Note: for an optimal algorithm we could also stop scanning 
                        #       the rows for this column
                        cols.remove(k)  

        return sorted(cols)
            
    
    
bool_cols = get_bool_cols('WDB_Case.csv')
print('Found', len(bool_cols), 'cols.', 'EXCERPT: ',)
print( ' '.join(bool_cols[:17]), '...')

from expected_bool_cols import expected_bool_cols
if len(bool_cols) != len(expected_bool_cols):
    print('ERROR! different lengths: bools_cols: %s    expected_bools_cols: %s' % (len(bool_cols), len(expected_bool_cols)))
else:
    for i in range(len(expected_bool_cols)):
        if bool_cols[i] != expected_bool_cols[i]:
            print('ERROR at index', i, ':')
            print('    ACTUAL:', repr(bool_cols[i]))
            print('  EXPECTED:', repr(expected_bool_cols[i]))               

## A2 fix_date

Implement `fix_date`, which takes a possibly partial date as a string `d/m/yyyy` and RETURN a string formatted as `mm/dd/yyyy`. If data is missing, omits it in the output as well, see examples.

In [3]:
def fix_date(d):
    
    if '/' in d:
        case_date = d.split('/')
        if len(case_date) == 3:
            d = case_date[1]
            m = case_date[0]
            y = case_date[2]
            return "%02d/%02d/%04d" % (int(d),int(m),int(y))
        elif len(case_date) == 2:
            m = case_date[0]
            y = case_date[1]
            return "%02d/%04d" % (int(m),int(y))                    
    return d
    

assert fix_date('2/8/1649') == '08/02/1649'
assert fix_date('25/4/1627') == '04/25/1627'
assert fix_date('6/11/1629') == '11/06/1629'
assert fix_date('12/1649') == '12/1649'
assert fix_date('7/1652') == '07/1652'
assert fix_date('1560') == '1560'
assert fix_date('') == ''
# Note there is a damned extra space in the dataset (Oswald,Katharine)
assert fix_date('13/11/ 1629') == '11/13/1629'

## A3 parse_db

Given a CSV of cases, outputs a list of dictionaries (parse with [CSV DictReader](https://en.softpython.org/formats/formats2-csv-sol.html#Reading-as-dictionaries)), each representing a case with these fields:

* `name`: the isolated name of the witch taken from `CaseCommonName` column if parseable, otherwise the full cell content
* `surname`: the isolated surname of the witch taken from `CaseCommonName` column if parseable, otherwise empty string
* `case_date`: `Case_date` column corrected with `fix_date`
* `suspects`: number of suspects as **integer**,  to be taken from the column `Suspects_text`. If column is empty, use `1`

`primary`, `secondary` and `tags` fields are to be filled with names of **boolean** columns for which the corresponding cell is marked with `'1'` according to these criteria:

* `primary`: a **single string** as follows: if a column ending with `_p` is marked `'1'`, this field contains that column name without the `'_p'`. If column name is `'NotEnoughInfo_p'` or in other cases, use `None`.
* `secondary`: **sorted** column names ending with `_s`. If col name is `'NotEnoughInfo_s'` or it's already present as `primary`, it's discarded. Remove trailing `_s` from values in the list.
* `tags`: **sorted** column names which are not primary nor secondary 


**Example** (full output is in [expected_cases_db.py](expected_cases_db.py)):

```python
>>> cases_db = parse_db('WDB_Case.csv')
>>> cases_db[1991:1994]
[{'primary': 'Demonic',
  'secondary': ['ImplicatedByAnother', 'Maleficium', 'UNorthodoxRelPract'],
  'tags': ['DevilPresent', 'UnorthodoxReligiousPractice', 'WitchesMeeting'],
  'name': 'Janet',
  'surname': 'McNicol',
  'suspects': 1,
  'case_date': '10/15/1673'},
 {'primary': None,
  'secondary': [],
  'tags': [],
  'name': 'Margaret',
  'surname': 'Clerk',
  'suspects': 1,
  'case_date': '06/04/1674'},
 {'primary': None,
  'secondary': ['Demonic'],
  'tags': ['Dancing', 'DevilPresent', 'Singing', 'WitchesMeeting'],
  'name': 'Agnes',
  'surname': 'Hendrie',
  'suspects': 1,
  'case_date': '07/29/1675'}]
```

In [4]:

import csv 

def parse_db(filename):
    
    
    characterisations = [c[:-2] for c in bool_cols if c.endswith('_p')]    
    other_bool_fields = [c for c in bool_cols if not (c.endswith('_p') or c.endswith('_s'))]    
    
    with open(filename, encoding='utf-8', newline='') as f:        
        my_reader = csv.DictReader(f, delimiter=',')  

        ret = []
        found = {}
        for diz in my_reader:
            work = {'primary': None,
                    'secondary':[],
                    'tags':[],
                   }
            for c in characterisations:
                if diz[c + '_p'] == '1':
                    if c != 'NotEnoughInfo':
                        work['primary'] = c  
                if diz[c + '_s'] == '1':
                    if c != 'NotEnoughInfo':
                        if c != work['primary']:
                            work['secondary'].append(c)                                   
                cn = diz['CaseCommonName']
                if ',' in cn:      
                    spli = cn.split(',')
                    work['name'] = spli[-1].strip()
                    if len(spli) == 2:
                        work['surname'] = spli[0].strip()
                    else:
                        work['surname'] = ','.join(spli[:-1]).strip()
                        
                else:
                    work['name'] = cn.strip()
                    work['surname'] = ''
                    
                sus = diz['Suspects_text']
                if sus:
                    work['suspects'] = int(sus)
                else:
                    work['suspects'] = 1
                    
            for c in other_bool_fields:
                if diz[c] == '1':
                    work['tags'].append(c)
                          
            work['case_date'] = fix_date(diz['Case_date'])            
            ret.append(work)
            
        return ret
    

cases_db = parse_db('WDB_Case.csv')

cases_db[1991:1994] 

In [5]:
# TESTS
assert cases_db[0]['primary'] == None
assert cases_db[0]['secondary'] == []
assert cases_db[0]['name'] == '3 unnamed witches'
assert cases_db[0]['surname'] == ''
assert cases_db[0]['suspects'] == 3  # int !
assert cases_db[0]['case_date'] == '08/02/1649'

assert cases_db[1]['primary'] == None
assert cases_db[1]['secondary'] ==  ['ImplicatedByAnother']
assert cases_db[1]['tags'] ==  []
assert cases_db[1]['name'] ==  'Cristine'
assert cases_db[1]['surname'] ==  'Kerington'
assert cases_db[1]['suspects'] ==  1    # Suspects_text is '', we put 1
assert cases_db[1]['case_date'] == '05/08/1591'

assert cases_db[1991]['primary'] == 'Demonic'
#NOTE: since 'Demonic' is already 'primary', we removed it from 'secondary'
assert cases_db[1991]['secondary'] == ['ImplicatedByAnother', 'Maleficium', 'UNorthodoxRelPract']
assert cases_db[1991]['tags'] == ['DevilPresent', 'UnorthodoxReligiousPractice', 'WitchesMeeting']
assert cases_db[1991]['name'] == 'Janet'
assert cases_db[1991]['surname'] == 'McNicol'
assert cases_db[1991]['suspects'] == 1    #  Suspects_text is '', we put 1
assert cases_db[1991]['case_date'] == '10/15/1673'

assert cases_db[0]['case_date'] == '08/02/1649'  # 2/8/1649
assert cases_db[1143]['case_date'] == '1560'     # 1560
assert cases_db[924]['case_date'] ==  '07/1652'  # 7/1652
assert cases_db[491]['suspects'] == 15           # 15

#composite name
assert cases_db[249]['name'] == 'Francis'     # "Stewart, Earl of Bothwell,Francis"
assert cases_db[249]['surname'] == 'Stewart, Earl of Bothwell'

from expected_cases_db import expected_cases_db
from pprint import pprint
for i in range(len(expected_cases_db)):
    if cases_db[i] != expected_cases_db[i]:    
        print('ERROR at index %s!' % i)
        print('ACTUAL:')
        pprint(cases_db[i])
        print('EXPECTED:', )
        pprint(expected_cases_db[i])
if len(cases_db) != len(expected_cases_db):
    print('ERROR! different lengths: cases_db: %s    expected_cases_db: %s' % (len(cases_db), len(expected_cases_db)))            
assert cases_db == expected_cases_db

## A4 plot_cases

Given the previously computed db, plot the number of cases per year.

* plot the ticks with 10 years intervals, according to the actual data (**DO NOT use constants like** `1560` **!!**)
* careful some cases have no year

![expected-plot.png](expected-plot.png)

In [6]:

def plot_cases(db):
    
    import numpy as np
    import matplotlib.pyplot as plt
    from collections import Counter

    years = [int(diz['case_date'].split('/')[-1]) for diz in db if diz['case_date']]
    hist = Counter(years)
    xs = sorted(hist.keys())    
    ys = [hist[x] for x in xs]
    
    fig = plt.figure(figsize=(18,5))
    plt.plot(xs, ys, color='purple')

    plt.title("Witchcraft cases per year SOLUTION")
    plt.xlabel('Year')
    plt.ylabel('Number of cases')

    tks = np.arange(min(xs), max(xs), 10)    
    plt.xticks(tks, tks)    
        
    plt.show()
    
    
plot_cases(cases_db)    

## Part B


## B1.1 Theory - Complexity

**Write the solution in separate `theory.txt` file**

1.  Given a list $L$ of $n$ elements, please compute the asymptotic computational complexity of the following function, explaining your reasoning.

```python

	def my_fun(L):
    		n = len(L)
    		if n <= 1:
        		return 1
    		else:
        		L1 = L[0:n//2]
        		L2 = L[n//2:]
        		a = my_fun(L1) + min(L1)
        		b = my_fun(L2) + min(L2)
        		return max(a, b)
```

## B1.2 Theory - Graph

Briefly describe what a BFS visit of a graph is. Please provide the order of visited nodes of a possible BFS visit on the following graph (starting from node 'a'):

![bfs](img/bfs.jpg)

## B2 Bank

<div class="alert alert-warning">
    
**WARNING: AN UPDATED VERSION OF THIS EXERCISE CAN BE** [FOUND HERE](https://sciprog.davidleoni.it/indexing/indexing.html#2.-Exercise---Bank)
</div>    

A bank stores all its transactions in a log: possible types of transactions are opening an account, transfering some amount of money, withdrawal, etc. For simplicity, we represent each transaction type with a character, and in the log which is a Python list we put only such characters (we don't care about their meaning).

The bank knows some sequences of events are extremely rare, and probably denote suspicious activity. For simplicitly, we assume such sequences only contain two transactions and represent them in tuples called _biseqs_. 


<div class="alert alert-warning">

**NOTE: a biseq can occur multiple times, but you can consider the number of repetitions negligible with respect to log size.**
</div>

**THESE EXERCISES ARE ABOUT CUSTOM INDEXING, NOT STRINGS: DO NOT** convert everything to a string, use regexes, nor use string methods like `.index()`, `.replace()`, etc

## B2.1 constructor,  log and pos
 
Implement following methods, adding some appropriate indexing data structure.

<div class="alert alert-warning">
    
**BEFORE IMPLEMENTING, DOUBLE CHECK COMPLEXITY REQUIREMENTS!**
    
The idea is that given _an entire biseq_ (not just the first character !), we want to find all of its (rare) occurrences _very fast_ in $O(1)$

Since performance is important here, passing tests does _not_ imply your work is correct!
    
</div>


 
```python 

class Bank:

    def __init__(self):
        """ Initializes an empty bank
        
            Add a data structure of your choice for indexing the log
        """
        self._trans = []
        raise Exception("TODO IMPLEMENT ME!")
        
    def log(self, transaction):
        """ Appends transaction to the log
        
            - REMEMBER to also update the index
            - MUST EXECUTE IN O(1)
        """
```        
 
```python
    def pos(self, biseq):
        """ RETURN a NEW list with all the indeces where the sequence of 
            two transactions can be found
        
            - MUST execute in O(1)
        """
```

**Testing**: `python3 -m unittest bank_test.LogTest`

**Example**:

In [7]:
from bank_sol import *

bank = Bank()
print(bank)

Bank: 


In [8]:
bank.log('a')

In [9]:
print(bank)

Bank: a


In [10]:
print(bank.pos( ('a','b') ))  # <-- a biseq is a tuple

[]


In [11]:
bank.log('b')

In [12]:
print(bank.pos( ('a','b') ))

[0]


In [13]:
bank.log('a')
bank.log('a')
bank.log('a')
bank.log('a')
bank.log('b')
print(bank)

Bank: a,b,a,a,a,a,b


In [14]:
print(bank.pos(('a','b')))

[0, 5]


In [15]:
print(bank.pos(('a','a')))

[2, 3, 4]


## B2.2 revert

```python
    def revert(self):
        """ Completely eliminates last transaction and RETURN it
        
            - if bank is empty, raises IndexError
            
            - REMEMBER to update any index referring to it
            - *MUST* EXECUTE IN O(1)             
        """
```
**Testing**: `python3 -m unittest RevertTest`

**Example:**

In [16]:
bank = Bank()
bank.log('a')
bank.log('b')
bank.log('c')
bank.log('a')
bank.log('b')
print(bank.pos(('a','b')))

[0, 3]


In [17]:
bank.revert()

'b'

In [18]:
print(bank)

Bank: a,b,c,a


In [19]:
print(bank.pos(('a','b')))

[0]


##  B2.3 max_interval

Typically, there is a biseq which triggers a period of suspicious activity, and there is another biseq which ends it. So given a start and an end biseq, the bank wants a report of all the transactions that happened in between:

```python
    def max_interval(self, bi_start, bi_end):
        """ RETURN a list with all the transactions occurred between 
            the *largest* interval among bi-sequences bi_start and bi_end
        
            - bi_start and bi_end are EXCLUDED 
            - if bi_start / bi_end are not found, or if bi_end is before/includes bi_start, 
              raise LookupError
            
            - DO *NOT* MODIFY the data structure
            - MUST EXECUTE IN O(k) where k is the length of the *largest* interval
              you can return
            - consider number of repetitions a negligible size
        """
```
**Testing**: `python -m unittest bank_test.MaxIntervalTest`

**Example**: 

In [20]:
bank = Bank()
bank.log('c')
bank.log('d')
bank.log('c')
bank.log('a')
bank.log('b') # <--- b 
bank.log('e') #      e
bank.log('f') #      --- f     |
bank.log('a') #          a     |
bank.log('f') #          f     | k
bank.log('c') #          c     |
bank.log('b') #      --- b     |
bank.log('a') # <--- a 
bank.log('f') #      f
bank.log('b')
bank.log('e')
bank.log('l')
bank.max_interval( ('b','e'), ('a','f') )

['f', 'a', 'f', 'c', 'b']

In [21]:
#ignore this
import jupman

import bank_test 
jupman.run(bank_test)

.................
----------------------------------------------------------------------
Ran 17 tests in 0.012s

OK
